---
jupyter: python3
---

This notebooks was used to find out which subreddits are most associated with each coin.

In [ ]:
import os
import praw
from collections import Counter
from datetime import datetime, timezone

from dotenv import load_dotenv, find_dotenv
import pandas as pd

# Load environment variables
dotenv_path = find_dotenv()
if dotenv_path and os.getenv("REDDIT_CLIENT_ID") is None:
    load_dotenv(dotenv_path)

# Initialize the Reddit API client
reddit = praw.Reddit(
    client_id=os.getenv("REDDIT_CLIENT_ID"),
    client_secret=os.getenv("REDDIT_CLIENT_SECRET"),
    user_agent=os.getenv("REDDIT_USER_AGENT"),
)

coins = pd.read_csv("../data/raw/labeled_coins.csv")

In [ ]:
list_coins = []
for _, coin in coins.iterrows():
    # Define your search term and date limit
    search_term = coin["Coin Name"]
    before_date = coin["Scam Date"]
    print(f"Searching for posts about '{search_term}' before {before_date}...")
    # Counter to store subreddit counts
    subreddit_counts = Counter()

    # Perform the search
    try:
        for submission in reddit.subreddit("all").search(search_term, time_filter="all", limit=10000):
            if str(before_date) == "nan":
                before_date = datetime.now().strftime("%Y-%m-%d")

            submission_date = datetime.fromtimestamp(submission.created_utc, timezone.utc).strftime("%Y-%m-%d")
            if submission_date <= before_date:  # Only include posts before the specified date
                subreddit_counts[submission.subreddit.display_name] += 1
    except Exception as e:
        print(f"An error occurred: {e}")

    # Get top subreddits by count
    top_subreddits = subreddit_counts.most_common(10)  # Adjust the number as needed
    print(f"Top subreddits for the coin '{search_term} until {before_date}':")
    temp = []
    for subreddit, count in top_subreddits:
        print(f"r/{subreddit}\t\t{count} results")
        temp.append(subreddit)

    list_coins.append({"coin": search_term, "subreddits": temp})

    print("\n")

In [ ]:
print(list_coins)